In [ ]:
import random
import cv2
import mediapipe as mp
import time
import numpy as np

# Initialize the camera
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Initialize mediapipe hands detector and drawing utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.8)
mp_drawing = mp.solutions.drawing_utils

# Initialize game variables
timer = 0
stateResult = False
startGame = False
scores = [0, 0]  # [AI, Player]

# Function to overlay a transparent PNG onto an image
def overlayPNG(background, overlay, position):
    y, x = position
    h, w = overlay.shape[:2]
    overlay_img = overlay[:, :, :3]  # RGB channels
    mask = overlay[:, :, 3:] / 255.0  # Alpha channel

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_img
    return background

# Main loop
while True:
    imgBG = cv2.imread("BG.png")  # Background image
    success, img = cap.read()

    # Flip the image for a mirror effect
    img = cv2.flip(img, 1)
    imgScaled = cv2.resize(img, (0, 0), None, 0.875, 0.875)
    imgScaled = imgScaled[:, 80:480]

    # Detect hand landmarks
    img_rgb = cv2.cvtColor(imgScaled, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    # Draw landmarks if detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(imgScaled, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # If game has started
    if startGame:
        if not stateResult:
            timer = time.time() - initialTime
            cv2.putText(imgBG, str(int(timer)), (605, 435), cv2.FONT_HERSHEY_PLAIN, 6, (255, 0, 255), 4)

            if timer > 1:
                stateResult = True
                timer = 0

                if results.multi_hand_landmarks:
                    playerMove = None
                    hand_landmarks = results.multi_hand_landmarks[0]

                    # Check finger positions for moves
                    fingers = []
                    if hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y:  # Index finger
                        fingers.append(1)
                    else:
                        fingers.append(0)
                    if hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y:  # Middle finger
                        fingers.append(1)
                    else:
                        fingers.append(0)
                    if hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y:  # Ring finger
                        fingers.append(1)
                    else:
                        fingers.append(0)
                    if hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y:  # Pinky finger
                        fingers.append(1)
                    else:
                        fingers.append(0)

                    # Map fingers up to moves (1: Rock, 2: Paper, 3: Scissors)
                    if fingers == [0, 0, 0, 0]:  # Rock
                        playerMove = 1
                    elif fingers == [1, 1, 1, 1]:  # Paper
                        playerMove = 2
                    elif fingers == [0, 1, 1, 0]:  # Scissors
                        playerMove = 3

                    # AI Move
                    randomNumber = random.randint(1, 3)
                    imgAI = cv2.imread(f'{randomNumber}.png', cv2.IMREAD_UNCHANGED)
                    imgBG = overlayPNG(imgBG, imgAI, (149, 310))

                    # Determine winner
                    if (playerMove == 1 and randomNumber == 3) or \
                            (playerMove == 2 and randomNumber == 1) or \
                            (playerMove == 3 and randomNumber == 2):
                        scores[1] += 1  # Player wins

                    elif (playerMove == 3 and randomNumber == 1) or \
                            (playerMove == 1 and randomNumber == 2) or \
                            (playerMove == 2 and randomNumber == 3):
                        scores[0] += 1  # AI wins

    # Display scores and scaled image on background
    imgBG[234:654, 795:1195] = imgScaled
    if stateResult:
        imgBG = overlayPNG(imgBG, imgAI, (310, 149)) # 149, 310

    # Display score
    cv2.putText(imgBG, str(scores[0]), (410, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 6)
    cv2.putText(imgBG, str(scores[1]), (1112, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 6)

    # Display final output
    cv2.imshow("BG", imgBG)

    # Start game on 's' key press
    key = cv2.waitKey(1)
    if key == ord('s'):
        startGame = True
        initialTime = time.time()
        stateResult = False

    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
